In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import logging
import os

# GPU 설정 - 7번 GPU만 사용하도록 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "7,9"

logging.basicConfig(level=logging.INFO)

# GPU 상태 확인
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Visible devices: {os.environ.get('CUDA_VISIBLE_DEVICES', 'Not set')}")
print(f"Device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

# 사용할 장치 설정: 환경 변수로 지정된 GPU의 0번 인덱스 사용
device = torch.device("cuda:0")  # CUDA_VISIBLE_DEVICES="7"로 설정했으므로 0번이 실제 7번 GPU
logging.info(f"Using device: {device}")

# bfloat16 지원 여부 확인
is_bfloat16 = torch.cuda.is_bf16_supported()
logging.info(f"bfloat16 supported: {is_bfloat16}")

# 모델 설정값
max_seq_len = 2048  # 긴 문맥 추론을 위해 증가 가능
logging.info(f"Max Sequence length set to: {max_seq_len}")

lora_rank = 16  # 클수록 성능이 좋지만 속도 저하

# 로컬 경로 설정
local_model_path = "/home/qudwo9246/GRPO/AtomicGPT-gemma2-9B"
local_tokenizer_path = "/home/qudwo9246/GRPO/AtomicGPT-gemma2-9B"

# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# 토크나이저 로드
try:
    tokenizer = AutoTokenizer.from_pretrained(local_tokenizer_path)
    logging.info("토크나이저 로드 완료")
except Exception as e:
    logging.error("토크나이저 로드 중 오류 발생:", exc_info=True)
    raise e

# 모델 로드
try:
    model = AutoModelForCausalLM.from_pretrained(
        local_model_path,
        quantization_config=bnb_config,  # 4bit 양자화 적용
        device_map="auto",  
        attn_implementation="eager"
    )
    model.config.max_position_embeddings = max_seq_len
    logging.info("모델 로드 완료")
except Exception as e:
    logging.error("모델 로드 중 오류 발생:", exc_info=True)
    raise e

# LoRA 설정 적용
peft_config = LoraConfig(
    r=lora_rank,
    lora_alpha=2*lora_rank,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# PEFT 적용
try:
    model = get_peft_model(model, peft_config)
    logging.info("PEFT 적용 완료")
except Exception as e:
    logging.error("PEFT 설정 중 오류 발생:", exc_info=True)
    raise e

# 모델 정보 출력
first_layer = model.base_model.model.model.layers[0].self_attn.q_proj
print(f"First layer weight dtype: {first_layer.weight.dtype}")
print(f"Model is quantized: {model.is_quantized}")

print("모델 및 LoRA 설정 완료!")
print("Max position embeddings:", model.config.max_position_embeddings)

In [4]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    """XML 태그에서 답변을 추출하고, 실패하면 다른 방법으로 시도합니다."""
    try:
        # 기본 XML 추출 시도
        if "<answer>" in text and "</answer>" in text:
            answer = text.split("<answer>")[-1]
            answer = answer.split("</answer>")[0]
            return answer.strip()
        
        # "Answer:" 키워드로 시도
        elif "Answer:" in text:
            answer = text.split("Answer:")[-1].strip()
            # 다음 줄바꿈이나 문장 끝까지 추출
            if "\n" in answer:
                answer = answer.split("\n")[0]
            return answer.strip()
        
        # 마지막 문장 시도 (최후의 수단)
        sentences = re.split(r'[.!?]', text)
        # 비어있지 않은 마지막 문장들 중 숫자가 포함된 것 찾기
        for sentence in reversed(sentences):
            if sentence.strip() and re.search(r'\d+', sentence):
                return sentence.strip()
        
        # 아무것도 찾지 못했으면 빈 문자열 반환
        return ""
    except Exception:
        return ""

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# 숫자 추출 및 비교를 위한 유틸리티 함수
def extract_final_number(text: str) -> float | None:
    """텍스트에서 가장 관련성 높은 숫자를 추출합니다."""
    if not text:
        return None
    
    # 결론 문장에서 숫자 찾기 시도
    conclusion_patterns = [
        r"Therefore,.*?(\d+)",
        r"Thus,.*?(\d+)",
        r"In total,.*?(\d+)",
        r"The answer is.*?(\d+)",
        r"The result is.*?(\d+)",
        r"equals.*?(\d+)"
    ]
    
    for pattern in conclusion_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                return float(match.group(1))
            except ValueError:
                pass
    
    # 모든 숫자 찾기
    numbers = re.findall(r'(\d+(?:\.\d+)?)', text)
    if numbers:
        try:
            # 마지막 숫자 반환 (일반적으로 최종 결과)
            return float(numbers[-1])
        except ValueError:
            pass
    
    return None

def compare_numbers(expected_str: str, actual_str: str) -> bool:
    """두 텍스트에서 숫자를 추출하여 비교합니다."""
    expected_num = extract_final_number(expected_str)
    actual_num = extract_final_number(actual_str)
    
    if expected_num is not None and actual_num is not None:
        # 소수점 이하 6자리까지 비교 (부동소수점 오차 허용)
        return abs(expected_num - actual_num) < 1e-6
    return False

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """모델의 출력에서 정답을 추출하여 비교하고 보상을 계산합니다."""
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content'] if prompts and len(prompts[0]) > 0 else "N/A"
    
    rewards = []
    
    for idx, (response, expected_answer) in enumerate(zip(responses, answer)):
        # 디버깅 정보 출력 (첫 번째 샘플만)
        if idx == 0:
            print('-'*20)
            print(f"Question:\n{q}")
            print(f"Expected Answer:\n{expected_answer if expected_answer else 'N/A'}")
            print(f"Full Response:\n{response if response else 'N/A'}")
        
        # 정답이 없으면 0점
        if not expected_answer:
            rewards.append(0.0)
            continue
        
        # 숫자 정답인 경우 숫자 추출
        expected_num = extract_final_number(expected_answer)
        
        # 1. 답변 추출 알고리즘을 사용하여 답변 추출
        extracted_answer = extract_xml_answer(response)
        
        if idx == 0:
            print(f"Extracted Answer: {extracted_answer}")
        
        # 추출된 답변이 없으면 0점
        if not extracted_answer:
            rewards.append(0.0)
            if idx == 0:
                print(f"✗ No answer extracted! Reward: 0.0")
            continue
        
        # 숫자 정답인 경우
        if expected_num is not None:
            actual_num = extract_final_number(extracted_answer)
            
            if idx == 0:
                print(f"Expected Number: {expected_num}, Extracted Number: {actual_num}")
            
            # 숫자가 정확히 일치하는 경우
            if actual_num is not None and abs(actual_num - expected_num) < 1e-6:
                # 추출된 답변이 숫자만 있거나 단위만 포함된 경우 (예: "110" 또는 "110 miles")
                if re.match(r'^\s*\$?\d+(\.\d+)?\s*$', extracted_answer) or re.match(r'^\s*\d+(\.\d+)?\s*[a-zA-Z]+\s*$', extracted_answer):
                    rewards.append(2.0)  # 정확한 숫자만 있으면 2.0 보상
                    if idx == 0:
                        print(f"✓ CORRECT with exact number! Reward: 2.0")
                else:
                    rewards.append(1.5)  # 정확한 숫자가 있지만 추가 텍스트도 있으면 1.5 보상
                    if idx == 0:
                        print(f"✓ CORRECT with additional text! Reward: 1.5")
            else:
                rewards.append(0.0)  # 숫자가 일치하지 않으면 0 보상
                if idx == 0:
                    print(f"✗ INCORRECT number! Reward: 0.0")
        
        # 숫자가 아닌 정답인 경우 (텍스트 비교)
        else:
            # 정답 텍스트 정규화 (대소문자, 공백 등 무시)
            normalized_expected = expected_answer.lower().strip()
            normalized_extracted = extracted_answer.lower().strip()
            
            if idx == 0:
                print(f"Text comparison - Expected: '{normalized_expected}', Actual: '{normalized_extracted}'")
            
            # 정확히 일치하는 경우
            if normalized_expected == normalized_extracted:
                rewards.append(2.0)
                if idx == 0:
                    print(f"✓ CORRECT with exact match! Reward: 2.0")
            # 부분 일치하는 경우 (핵심 키워드가 포함된 경우)
            elif normalized_expected in normalized_extracted or normalized_extracted in normalized_expected:
                rewards.append(1.5)
                if idx == 0:
                    print(f"✓ CORRECT with partial match! Reward: 1.5")
            else:
                rewards.append(0.0)
                if idx == 0:
                    print(f"✗ INCORRECT text! Reward: 0.0")
    
    return rewards

def int_reward_func(completions, **kwargs) -> list[float]:
    """출력에 숫자가 포함되어 있으면 보상을 줍니다."""
    responses = [completion[0]['content'] for completion in completions]
    
    # 전체 응답에서 숫자 찾기
    rewards = []
    for r in responses:
        if re.search(r'\d+', r):
            rewards.append(0.5)
        else:
            rewards.append(0.0)
    
    return rewards

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """응답이 정확한 XML 형식을 따르는지 확인하고 보상을 줍니다."""
    # 정확한 XML 형식 패턴
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    
    rewards = []
    for r in responses:
        if re.search(pattern, r, re.DOTALL):
            rewards.append(0.5)
        else:
            rewards.append(0.0)
    
    return rewards

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """응답이 유연한 형식(XML 또는 키워드)을 따르는지 확인하고 보상을 줍니다."""
    # XML 형식 또는 키워드 형식 패턴
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    
    responses = [completion[0]["content"] for completion in completions]
    
    rewards = []
    for r in responses:
        if re.search(pattern, r, re.DOTALL):
            rewards.append(0.25)
        else:
            rewards.append(0.0)
    
    return rewards

def count_xml(text) -> float:
    """XML 태그 또는 키워드의 존재 여부에 따라 보상을 계산합니다."""
    if not text:
        return 0.0
        
    count = 0.0
    # XML 태그 확인
    if "<reasoning>" in text:
        count += 0.125
    if "</reasoning>" in text:
        count += 0.125
    if "<answer>" in text:
        count += 0.125
    if "</answer>" in text:
        count += 0.125
    
    # 대체 형식 확인 (XML 태그가 없는 경우에도 일부 보상)
    if "Reasoning:" in text:
        count += 0.0625
    if "Answer:" in text:
        count += 0.0625
        
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """응답에서 XML 태그 또는 키워드의 존재 여부를 측정하여 보상합니다."""
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [8]:
import os
import torch
from trl import GRPOConfig, GRPOTrainer

# GPU와 CUDA 환경 설정
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7, 9" # 7번 GPU 사용

# bfloat16 지원 여부 확인
bf16_supported = torch.cuda.is_bf16_supported()

training_args = GRPOConfig(
use_vllm=False, # ✅ vLLM 지원 여부 불확실 → False로 변경
learning_rate=5e-6,
adam_beta1=0.9,
adam_beta2=0.99,
weight_decay=0.1,
warmup_ratio=0.1,
lr_scheduler_type="cosine",
optim="adamw_bnb_8bit",
logging_steps=1,
bf16=True, # ✅ bfloat16이 가능하면 사용
fp16=False, # ✅ fp16 사용 안 함 (오버플로우 방지)
per_device_train_batch_size=4, # ✅ batch size 감소 (OOM 방지)
gradient_accumulation_steps=1, # ✅ grad accumulation 추가
num_generations=4,
max_prompt_length=256,
max_completion_length=250,
max_steps=1000,
save_steps=250,
max_grad_norm=0.1,
report_to="none",
output_dir="outputs",
do_train=True,
# 멀티 GPU 관련 설정 추가
ddp_find_unused_parameters=False,
dataloader_num_workers=2,
)

In [ ]:
# 모델을 bfloat16으로 변환
model = model.to(torch.bfloat16)
print(f"모델 데이터 타입: {next(model.parameters()).dtype}")

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()